# Creating a new planner in nuPlan <a name="introduction"></a>


## Setup

In [1]:
# Useful imports
import os
import hydra
import nest_asyncio
from IPython.core.display import display, HTML
from bokeh.io import output_notebook


/tmp/ipykernel_14008/4095267831.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
nest_asyncio.apply()
output_notebook()
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

Loading BokehJS ...

# Simulating the planner <a name="simulation"></a>

In [3]:
%pwd

'/host_home/Repos/nuplan-devkit/experiments'

In [4]:
%env NUPLAN_DATA_ROOT=/home/ehdykhne/Repos/Datasets/nuplan/dataset
%env NUPLAN_MAPS_ROOT=/home/ehdykhne/Repos/Datasets/nuplan/dataset/maps

env: NUPLAN_DATA_ROOT=/home/ehdykhne/Repos/Datasets/nuplan/dataset
env: NUPLAN_MAPS_ROOT=/home/ehdykhne/Repos/Datasets/nuplan/dataset/maps


## Prepare the simulation config

In [5]:
from tutorials.utils.tutorial_utils import construct_simulation_hydra_paths

# Location of paths with all simulation configs
BASE_CONFIG_PATH = os.path.join(os.getenv('NUPLAN_TUTORIAL_PATH', ''), '../nuplan/planning/script')
simulation_hydra_paths = construct_simulation_hydra_paths(BASE_CONFIG_PATH)

# Create a temporary directory to store the simulation artifacts

DATASET_PARAMS = [
    'scenario_builder=nuplan_mini',  # use nuplan mini database (2.5h of 8 autolabeled logs in Las Vegas)
    'scenario_filter=one_continuous_log',  # simulate only one log
    "scenario_filter.log_names=['2021.06.14.16.48.02_veh-12_04057_04438']",
    'scenario_filter.limit_total_scenarios=1',  # use 1 total scenarios
]
ckpt_dir = '/home/ehdykhne/Repos/nuplan-devkit/experiments/pretrained_checkpoints/pdm_offset_checkpoint.ckpt'
#'/home/sacardoz/checkpoints/urbandriver_checkpoint.ckpt'
#"/home/sacardoz/tutorial_vector_framework/training_simple_vector_experiment/train_default_simple_vector/2023.11.23.09.55.21/best_model/epoch.ckpt"
#"/home/sacardoz/training_raster_experiment/train_default_raster/2023.11.23.07.36.36/best_model/epoch.ckpt"
# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=simulation_hydra_paths.config_path)

# Compose the configuration
cfg = hydra.compose(config_name=simulation_hydra_paths.config_name, overrides=[
    '+simulation=closed_loop_reactive_agents',
    #'model=pgm_hybrid_model',
    'planner=pdm_hybrid_planner',
    f"planner.pdm_hybrid_planner.checkpoint_path={ckpt_dir}" ,
    #'planner.ml_planner.model_config=${model}',
    #f'planner.ml_planner.checkpoint_path={ckpt_dir}',
    #f'observation=idm_agents_observation',
    #'observation.model_config=${model}',
    #f'observation.checkpoint_path={ckpt_dir}',
    'worker=sequential',
    '+occlusion=true',
    '+occlusion.manager_type=wedge', #options: [range, shadow, wedge]
    "hydra.searchpath=[pkg://tuplan_garage.planning.script.config.common, pkg://tuplan_garage.planning.script.config.simulation, pkg://nuplan.planning.script.config.common, pkg://nuplan.planning.script.experiments]",
    *DATASET_PARAMS,
])

## Launch simulation (within the notebook)

In [6]:
from nuplan.planning.script.run_simulation import build_simulation_runners
from nuplan.common.actor_state.tracked_objects_types import AGENT_TYPES, STATIC_OBJECT_TYPES, TrackedObjectType

# Run the simulation loop (real-time visualization not yet supported, see next section for visualization)
runners, common_builder, cfg = build_simulation_runners(cfg)

Global seed set to 0
INFO:nuplan.planning.script.builders.main_callback_builder:Building MultiMainCallback...
INFO:nuplan.planning.script.builders.main_callback_builder:Building MultiMainCallback: 4...DONE!


2023-12-04 03:33:03,757 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:19}  Building WorkerPool...
2023-12-04 03:33:03,758 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:101}  Worker: Sequential
2023-12-04 03:33:03,758 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:102}  Number of nodes: 1
Number of CPUs per node: 1
Number of GPUs per node: 0
Number of threads across all nodes: 1
2023-12-04 03:33:03,759 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:27}  Building WorkerPool...DONE!
2023-12-04 03:33:03,759 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:32}  Building experiment folders...
2023-12-04 03:33:03,759 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:35}  

	Folder where all results are stored: /home/ehdykh

In [7]:
runner = runners[0]

In [8]:
runner.simulation.callback.on_simulation_start(runner.simulation.setup)

# Initialize all simulations
runner._initialize()

while runner.simulation.is_simulation_running():
    # Execute specific callback
    runner.simulation.callback.on_step_start(runner.simulation.setup, runner.planner)

    # Perform step
    planner_input = runner._simulation.get_planner_input()

    # Execute specific callback
    runner._simulation.callback.on_planner_start(runner.simulation.setup, runner.planner)

    # Plan path based on all planner's inputs
    trajectory = runner.planner.compute_trajectory(planner_input)

    # Propagate simulation based on planner trajectory
    runner._simulation.callback.on_planner_end(runner.simulation.setup, runner.planner, trajectory)

    iteration = runner.simulation._time_controller.get_iteration()

    runner.simulation.propagate(trajectory)

    # Execute specific callback
    runner.simulation.callback.on_step_end(runner.simulation.setup, runner.planner, runner.simulation.history.last())

runner.simulation.callback.on_simulation_end(runner.simulation.setup, runner.planner, runner.simulation.history)

elapsed time: 0.03236079216003418
elapsed time: 0.03239083290100098
elapsed time: 0.03203535079956055
elapsed time: 0.03168797492980957
elapsed time: 0.03194427490234375
elapsed time: 0.03200364112854004
elapsed time: 0.03193473815917969
elapsed time: 0.033086538314819336
elapsed time: 0.03194141387939453
elapsed time: 0.031324148178100586
elapsed time: 0.03136634826660156
elapsed time: 0.03155016899108887
elapsed time: 0.03175497055053711
elapsed time: 0.0321657657623291
elapsed time: 0.03327345848083496
elapsed time: 0.03334665298461914
elapsed time: 0.031965017318725586
elapsed time: 0.03251028060913086
elapsed time: 0.031733036041259766
elapsed time: 0.031287431716918945
elapsed time: 0.030990123748779297
elapsed time: 0.032648563385009766
elapsed time: 0.03125643730163574
elapsed time: 0.030818700790405273
elapsed time: 0.031053543090820312
elapsed time: 0.031400203704833984
elapsed time: 0.031803131103515625
elapsed time: 0.03257393836975098
elapsed time: 0.03282809257507324
elap

In [9]:
from tutorials.utils.tutorial_utils import visualize_history
visualize_history(runner.simulation._history, runner.scenario, bokeh_port=5006)

INFO:bokeh.server.server:Starting Bokeh server version 2.4.3 (running on Tornado 6.2)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


2023-12-04 03:34:54,277 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/nuboard/base/experiment_file_data.py:140}  Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
2023-12-04 03:34:54,278 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/nuboard/base/experiment_file_data.py:140}  Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
2023-12-04 03:34:54,278 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/nuboard/base/experiment_file_data.py:140}  Error creating dataset. Could not read schema from 'pretrained_checkpoints/gc_pgp_checkpoint.ckpt'. Is this a 'parquet' file?: Could not open Parquet input source 'pretrained_checkpoints/gc_pgp_checkpoint.ckpt': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
2023-12-04 03:34:

Rendering a scenario: 100%|██████████| 1/1 [00:00<00:00, 151.59it/s]


2023-12-04 03:34:56,502 INFO {/opt/conda/lib/python3.9/site-packages/tornado/web.py:2271}  200 GET / (10.40.115.212) 2235.79ms
2023-12-04 03:34:56,511 INFO {/home/ehdykhne/Repos/nuplan-devkit/tutorials/utils/tutorial_utils.py:267}  Done rendering!


INFO:tornado.access:200 GET / (10.40.115.212) 2235.79ms
INFO:bokeh.server.views.ws:WebSocket connection opened
INFO:tornado.access:101 GET /ws (10.40.115.212) 0.49ms
INFO:bokeh.server.views.ws:ServerConnection created


2023-12-04 03:34:57,468 INFO {/opt/conda/lib/python3.9/site-packages/tornado/web.py:2271}  101 GET /ws (10.40.115.212) 0.49ms
